# EEG and EMG Channel Extraction and Preprocessing - C067 Subject

This notebook extracts EEG and EMG channels from C067_EDF_Export.edf and applies standard preprocessing filters:
- **EEG**: 0.5-40 Hz bandpass + 50Hz notch filters (and harmonics)
- **EMG**: 20-500 Hz bandpass + 50Hz notch filters (and harmonics)


In [4]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import mne
from pathlib import Path
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12


In [5]:
# Load the EDF file
data_dir = "/Users/yangqi/Desktop/RBD/luzy_raw_data"
c067_file = Path(data_dir) / "C067" / "C067_EDF_Export.edf"

print(f"Loading EDF file: {c067_file}")
print(f"File exists: {c067_file.exists()}")

# Read the EDF file
raw = mne.io.read_raw_edf(str(c067_file), preload=True, verbose=False)
print(f"\nFile loaded successfully!")
print(f"Number of channels: {len(raw.ch_names)}")
print(f"Sampling frequency: {raw.info['sfreq']} Hz")
print(f"Duration: {raw.times[-1]:.2f} seconds")

# Store sampling frequency
sfreq = raw.info['sfreq']


Loading EDF file: /Users/yangqi/Desktop/RBD/luzy_raw_data/C067/C067_EDF_Export.edf
File exists: True

File loaded successfully!
Number of channels: 35
Sampling frequency: 1024.0 Hz
Duration: 36454.00 seconds


In [7]:
# Display all channels and identify EEG/EMG channels
print("=" * 70)
print("CHANNEL IDENTIFICATION")
print("=" * 70)
print("\nAll available channels:")
for i, ch_name in enumerate(raw.ch_names):
    print(f"{i+1:2d}. {ch_name}")

# Define standard EEG channel names (10-20 system)
eeg_channel_names = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2', 'M1', 'M2', 'E1', 'E2']

# Define EMG channel names (common patterns)
emg_channel_patterns = ['Chin', 'chin', 'EMG', 'emg', 'Leg', 'leg']

# Find EEG channels
eeg_channels = []
for ch_name in raw.ch_names:
    if ch_name in eeg_channel_names:
        eeg_channels.append(ch_name)

# Find EMG channels
emg_channels = []
for ch_name in raw.ch_names:
    # Check if channel name contains EMG patterns
    if any(pattern.lower() in ch_name.lower() for pattern in emg_channel_patterns):
        emg_channels.append(ch_name)

print(f"\n" + "=" * 70)
print(f"EEG Channels Found: {len(eeg_channels)}")
print("=" * 70)
if eeg_channels:
    for ch in eeg_channels:
        print(f"  - {ch}")
else:
    print("  No EEG channels found!")

print(f"\n" + "=" * 70)
print(f"EMG Channels Found: {len(emg_channels)}")
print("=" * 70)
if emg_channels:
    for ch in emg_channels:
        print(f"  - {ch}")
else:
    print("  No EMG channels found!")

print("=" * 70)


CHANNEL IDENTIFICATION

All available channels:
 1. F3
 2. F4
 3. C3
 4. C4
 5. M1
 6. M2
 7. O1
 8. O2
 9. E1
10. E2
11. Chin 1
12. Chin 2
13. Chin 3
14. ECG_Spare
15. Leg/L
16. Leg/R
17. ECG
18. Therm
19. Resp Imp
20. Thor
21. Abdo
22. Nasal Pressure
23. Light
24. PositionSen
25. Snore
26. SpO2
27. Pulse
28. Ox Status
29. Pleth
30. Derived HR
31. ManPosition
32. ManLights
33. Respiratory Rate
34. DHR
35. Sum Effort

EEG Channels Found: 10
  - F3
  - F4
  - C3
  - C4
  - M1
  - M2
  - O1
  - O2
  - E1
  - E2

EMG Channels Found: 5
  - Chin 1
  - Chin 2
  - Chin 3
  - Leg/L
  - Leg/R


In [8]:
# Extract EEG and EMG data
# Extract first 100 seconds for analysis (can be modified)
start_time = 0
end_time = 100.0
start_sample = int(start_time * sfreq)
end_sample = int(end_time * sfreq)

print("=" * 70)
print("DATA EXTRACTION")
print("=" * 70)
print(f"\nExtracting data from {start_time} to {end_time} seconds")

# Extract EEG data
eeg_data = None
eeg_channel_indices = []
if eeg_channels:
    eeg_channel_indices = [raw.ch_names.index(ch) for ch in eeg_channels]
    eeg_data = raw.get_data(picks=eeg_channel_indices, start=start_sample, stop=end_sample)
    print(f"\nEEG data extracted:")
    print(f"  Channels: {eeg_channels}")
    print(f"  Shape: {eeg_data.shape}")
    print(f"  Sampling frequency: {sfreq} Hz")

# Extract EMG data
emg_data = None
emg_channel_indices = []
if emg_channels:
    emg_channel_indices = [raw.ch_names.index(ch) for ch in emg_channels]
    emg_data = raw.get_data(picks=emg_channel_indices, start=start_sample, stop=end_sample)
    print(f"\nEMG data extracted:")
    print(f"  Channels: {emg_channels}")
    print(f"  Shape: {emg_data.shape}")
    print(f"  Sampling frequency: {sfreq} Hz")

# Create time axis
if eeg_data is not None:
    time_axis = np.linspace(start_time, end_time, eeg_data.shape[1])
elif emg_data is not None:
    time_axis = np.linspace(start_time, end_time, emg_data.shape[1])

print("=" * 70)


DATA EXTRACTION

Extracting data from 0 to 100.0 seconds

EEG data extracted:
  Channels: ['F3', 'F4', 'C3', 'C4', 'M1', 'M2', 'O1', 'O2', 'E1', 'E2']
  Shape: (10, 102400)
  Sampling frequency: 1024.0 Hz

EMG data extracted:
  Channels: ['Chin 1', 'Chin 2', 'Chin 3', 'Leg/L', 'Leg/R']
  Shape: (5, 102400)
  Sampling frequency: 1024.0 Hz


In [9]:
# EEG Filtering: 0.5-40 Hz bandpass + 50Hz notch filters (and harmonics)
if eeg_data is not None:
    print("=" * 70)
    print("EEG FILTERING PIPELINE")
    print("=" * 70)
    
    # EEG filter parameters
    highpass_freq = 0.5  # High-pass cutoff (Hz)
    lowpass_freq = 40.0  # Low-pass cutoff (Hz)
    notch_quality = 30.0  # Notch quality factor
    
    # Notch frequencies: 50Hz and multiples up to 250Hz (within EEG range)
    notch_frequencies_eeg = np.arange(50, 251, 50)  # 50, 100, 150, 200, 250 Hz
    
    print(f"\nFilter parameters:")
    print(f"  Bandpass: {highpass_freq}-{lowpass_freq} Hz")
    print(f"  Notch frequencies: {notch_frequencies_eeg} Hz")
    print(f"  Sampling frequency: {sfreq} Hz")
    
    # Initialize filtered data
    eeg_data_filtered = eeg_data.copy()
    nyquist = sfreq / 2
    
    # Step 1: Bandpass filter (0.5-40 Hz)
    print(f"\nStep 1: Applying bandpass filter ({highpass_freq}-{lowpass_freq} Hz)...")
    low_norm = highpass_freq / nyquist
    high_norm = lowpass_freq / nyquist
    b_bp, a_bp = signal.butter(4, [low_norm, high_norm], btype='band')
    
    for i in range(eeg_data_filtered.shape[0]):
        eeg_data_filtered[i, :] = signal.filtfilt(b_bp, a_bp, eeg_data_filtered[i, :])
    print(f"  ✓ Bandpass filter applied to {eeg_data_filtered.shape[0]} channels")
    
    # Step 2: Notch filters (50Hz and multiples)
    print(f"\nStep 2: Applying notch filters...")
    for notch_freq in notch_frequencies_eeg:
        if notch_freq < nyquist:
            b_notch, a_notch = signal.iirnotch(notch_freq, notch_quality, sfreq)
            for i in range(eeg_data_filtered.shape[0]):
                eeg_data_filtered[i, :] = signal.filtfilt(b_notch, a_notch, eeg_data_filtered[i, :])
            print(f"  ✓ Notch filter applied at {notch_freq} Hz")
    
    print(f"\n" + "=" * 70)
    print("EEG FILTERING COMPLETE")
    print("=" * 70)
    
    # Display statistics
    print(f"\nFiltering effects (first channel {eeg_channels[0]}):")
    original_std = np.std(eeg_data[0, :])
    filtered_std = np.std(eeg_data_filtered[0, :])
    print(f"  Original std: {original_std:.6f}")
    print(f"  Filtered std: {filtered_std:.6f}")
    print(f"  Reduction: {(1 - filtered_std/original_std)*100:.1f}%")
    print("=" * 70)
else:
    print("No EEG data to filter")


EEG FILTERING PIPELINE

Filter parameters:
  Bandpass: 0.5-40.0 Hz
  Notch frequencies: [ 50 100 150 200 250] Hz
  Sampling frequency: 1024.0 Hz

Step 1: Applying bandpass filter (0.5-40.0 Hz)...
  ✓ Bandpass filter applied to 10 channels

Step 2: Applying notch filters...
  ✓ Notch filter applied at 50 Hz
  ✓ Notch filter applied at 100 Hz
  ✓ Notch filter applied at 150 Hz
  ✓ Notch filter applied at 200 Hz
  ✓ Notch filter applied at 250 Hz

EEG FILTERING COMPLETE

Filtering effects (first channel F3):
  Original std: 0.000956
  Filtered std: 0.000088
  Reduction: 90.8%


In [10]:
# EMG Filtering: 20-500 Hz bandpass + 50Hz notch filters (and harmonics)
if emg_data is not None:
    print("=" * 70)
    print("EMG FILTERING PIPELINE")
    print("=" * 70)
    
    # EMG filter parameters
    lowcut = 20.0  # High-pass cutoff (Hz)
    highcut = 500.0  # Low-pass cutoff (Hz)
    notch_quality = 30.0  # Notch quality factor
    
    # Notch frequencies: 50Hz and multiples within EMG range (20-500 Hz)
    notch_frequencies_emg = np.arange(50, 501, 50)  # 50, 100, 150, ..., 500 Hz
    
    print(f"\nFilter parameters:")
    print(f"  Bandpass: {lowcut}-{highcut} Hz")
    print(f"  Notch frequencies: {notch_frequencies_emg} Hz")
    print(f"  Sampling frequency: {sfreq} Hz")
    
    # Initialize filtered data
    emg_data_filtered = emg_data.copy()
    nyquist = sfreq / 2
    
    # Step 1: Bandpass filter (20-500 Hz)
    print(f"\nStep 1: Applying bandpass filter ({lowcut}-{highcut} Hz)...")
    low_norm = lowcut / nyquist
    high_norm = highcut / nyquist
    b_bp, a_bp = signal.butter(4, [low_norm, high_norm], btype='band')
    
    for i in range(emg_data_filtered.shape[0]):
        emg_data_filtered[i, :] = signal.filtfilt(b_bp, a_bp, emg_data_filtered[i, :])
    print(f"  ✓ Bandpass filter applied to {emg_data_filtered.shape[0]} channels")
    
    # Step 2: Notch filters (50Hz and multiples)
    print(f"\nStep 2: Applying notch filters...")
    for notch_freq in notch_frequencies_emg:
        if lowcut < notch_freq < highcut and notch_freq < nyquist:
            b_notch, a_notch = signal.iirnotch(notch_freq, notch_quality, sfreq)
            for i in range(emg_data_filtered.shape[0]):
                emg_data_filtered[i, :] = signal.filtfilt(b_notch, a_notch, emg_data_filtered[i, :])
            print(f"  ✓ Notch filter applied at {notch_freq} Hz")
    
    print(f"\n" + "=" * 70)
    print("EMG FILTERING COMPLETE")
    print("=" * 70)
    
    # Display statistics
    print(f"\nFiltering effects (first channel {emg_channels[0]}):")
    original_std = np.std(emg_data[0, :])
    filtered_std = np.std(emg_data_filtered[0, :])
    print(f"  Original std: {original_std:.6f}")
    print(f"  Filtered std: {filtered_std:.6f}")
    print(f"  Reduction: {(1 - filtered_std/original_std)*100:.1f}%")
    print("=" * 70)
else:
    print("No EMG data to filter")


EMG FILTERING PIPELINE

Filter parameters:
  Bandpass: 20.0-500.0 Hz
  Notch frequencies: [ 50 100 150 200 250 300 350 400 450 500] Hz
  Sampling frequency: 1024.0 Hz

Step 1: Applying bandpass filter (20.0-500.0 Hz)...
  ✓ Bandpass filter applied to 5 channels

Step 2: Applying notch filters...
  ✓ Notch filter applied at 50 Hz
  ✓ Notch filter applied at 100 Hz
  ✓ Notch filter applied at 150 Hz
  ✓ Notch filter applied at 200 Hz
  ✓ Notch filter applied at 250 Hz
  ✓ Notch filter applied at 300 Hz
  ✓ Notch filter applied at 350 Hz
  ✓ Notch filter applied at 400 Hz
  ✓ Notch filter applied at 450 Hz

EMG FILTERING COMPLETE

Filtering effects (first channel Chin 1):
  Original std: 0.000960
  Filtered std: 0.000118
  Reduction: 87.7%


In [12]:
# Summary: Display all extracted and filtered channels
print("=" * 70)
print("PREPROCESSING SUMMARY")
print("=" * 70)

if eeg_data is not None:
    print(f"\nEEG Channels ({len(eeg_channels)}):")
    print(f"  Channels: {eeg_channels}")
    print(f"  Data shape: {eeg_data.shape}")
    print(f"  Filtered data shape: {eeg_data_filtered.shape}")
    print(f"  Filter: 0.5-40 Hz bandpass + notch filters (50, 100, 150, 200, 250 Hz)")

if emg_data is not None:
    print(f"\nEMG Channels ({len(emg_channels)}):")
    print(f"  Channels: {emg_channels}")
    print(f"  Data shape: {emg_data.shape}")
    print(f"  Filtered data shape: {emg_data_filtered.shape}")
    print(f"  Filter: 20-500 Hz bandpass + notch filters (50, 100, 150, ..., 500 Hz)")

print(f"\n" + "=" * 70)
print("All data extracted and filtered successfully!")
print("=" * 70)
print("\nVariables available:")
if eeg_data is not None:
    print("  - eeg_data: Original EEG data")
    print("  - eeg_data_filtered: Filtered EEG data")
    print("  - eeg_channels: List of EEG channel names")
if emg_data is not None:
    print("  - emg_data: Original EMG data")
    print("  - emg_data_filtered: Filtered EMG data")
    print("  - emg_channels: List of EMG channel names")
print("  - time_axis: Time axis for plotting")
print("  - sfreq: Sampling frequency")
print("=" * 70)


PREPROCESSING SUMMARY

EEG Channels (10):
  Channels: ['F3', 'F4', 'C3', 'C4', 'M1', 'M2', 'O1', 'O2', 'E1', 'E2']
  Data shape: (10, 102400)
  Filtered data shape: (10, 102400)
  Filter: 0.5-40 Hz bandpass + notch filters (50, 100, 150, 200, 250 Hz)

EMG Channels (5):
  Channels: ['Chin 1', 'Chin 2', 'Chin 3', 'Leg/L', 'Leg/R']
  Data shape: (5, 102400)
  Filtered data shape: (5, 102400)
  Filter: 20-500 Hz bandpass + notch filters (50, 100, 150, ..., 500 Hz)

All data extracted and filtered successfully!

Variables available:
  - eeg_data: Original EEG data
  - eeg_data_filtered: Filtered EEG data
  - eeg_channels: List of EEG channel names
  - emg_data: Original EMG data
  - emg_data_filtered: Filtered EMG data
  - emg_channels: List of EMG channel names
  - time_axis: Time axis for plotting
  - sfreq: Sampling frequency


In [13]:
# Read XML annotations and align with PSG data
import xml.etree.ElementTree as ET
import pandas as pd

# Path to XML file
xml_file = Path(data_dir) / "C067" / "C067_EDF_Export.edf.XML"

print("=" * 70)
print("XML ANNOTATION READING")
print("=" * 70)
print(f"\nLoading XML file: {xml_file}")
print(f"File exists: {xml_file.exists()}")

if xml_file.exists():
    # Parse XML file
    tree = ET.parse(str(xml_file))
    root = tree.getroot()
    
    # Extract epoch length
    epoch_length = float(root.find('EpochLength').text) if root.find('EpochLength') is not None else 30.0
    print(f"\nEpoch length: {epoch_length} seconds")
    
    # Extract sleep stages
    sleep_stages = []
    sleep_stages_elem = root.find('SleepStages')
    if sleep_stages_elem is not None:
        for stage in sleep_stages_elem.findall('SleepStage'):
            sleep_stages.append(int(stage.text))
        print(f"Sleep stages found: {len(sleep_stages)} epochs")
        print(f"  Total duration: {len(sleep_stages) * epoch_length / 3600:.2f} hours")
    
    # Create sleep stage time axis (in seconds)
    if sleep_stages:
        sleep_stage_times = np.arange(0, len(sleep_stages) * epoch_length, epoch_length)
        print(f"  Time range: 0 - {sleep_stage_times[-1] + epoch_length:.1f} seconds")
    
    # Extract scored events
    scored_events = []
    scored_events_elem = root.find('ScoredEvents')
    if scored_events_elem is not None:
        for event in scored_events_elem.findall('ScoredEvent'):
            event_dict = {}
            event_dict['Name'] = event.find('Name').text if event.find('Name') is not None else None
            event_dict['Start'] = float(event.find('Start').text) if event.find('Start') is not None else None
            event_dict['Duration'] = float(event.find('Duration').text) if event.find('Duration') is not None else None
            event_dict['Input'] = event.find('Input').text if event.find('Input') is not None else None
            
            # Additional fields
            if event.find('LowestSpO2') is not None:
                event_dict['LowestSpO2'] = int(event.find('LowestSpO2').text)
            if event.find('Desaturation') is not None:
                event_dict['Desaturation'] = int(event.find('Desaturation').text)
            
            scored_events.append(event_dict)
        
        print(f"\nScored events found: {len(scored_events)} events")
        
        # Count events by type
        event_types = {}
        for event in scored_events:
            event_name = event.get('Name', 'Unknown')
            event_types[event_name] = event_types.get(event_name, 0) + 1
        
        print(f"\nEvent types summary:")
        for event_name, count in sorted(event_types.items(), key=lambda x: x[1], reverse=True):
            print(f"  {event_name}: {count}")
    
    # Create DataFrame for easier analysis
    if scored_events:
        events_df = pd.DataFrame(scored_events)
        print(f"\nEvents DataFrame created: {events_df.shape}")
        print(f"\nFirst few events:")
        print(events_df.head(10).to_string())
    
    # Align annotations with extracted data time range
    print(f"\n" + "=" * 70)
    print("ANNOTATION ALIGNMENT WITH EXTRACTED DATA")
    print("=" * 70)
    print(f"\nExtracted data time range: {start_time} - {end_time} seconds")
    
    # Filter sleep stages within extracted time range
    if sleep_stages:
        start_epoch = int(start_time / epoch_length)
        end_epoch = int(end_time / epoch_length)
        aligned_sleep_stages = sleep_stages[start_epoch:end_epoch]
        aligned_sleep_stage_times = sleep_stage_times[start_epoch:end_epoch]
        
        print(f"\nAligned sleep stages:")
        print(f"  Epochs in range: {len(aligned_sleep_stages)}")
        print(f"  Time range: {aligned_sleep_stage_times[0]:.1f} - {aligned_sleep_stage_times[-1] + epoch_length:.1f} seconds")
        
        # Sleep stage distribution
        stage_labels = {0: 'Wake', 1: 'N1', 2: 'N2', 3: 'N3', 5: 'REM'}
        stage_counts = {}
        for stage in aligned_sleep_stages:
            stage_name = stage_labels.get(stage, f'Unknown({stage})')
            stage_counts[stage_name] = stage_counts.get(stage_name, 0) + 1
        
        print(f"\n  Sleep stage distribution in extracted range:")
        for stage_name, count in sorted(stage_counts.items()):
            duration_min = count * epoch_length / 60
            print(f"    {stage_name}: {count} epochs ({duration_min:.1f} minutes)")
    
    # Filter events within extracted time range
    if scored_events:
        aligned_events = [e for e in scored_events 
                         if e.get('Start') is not None 
                         and start_time <= e['Start'] < end_time]
        
        print(f"\nAligned scored events:")
        print(f"  Events in range: {len(aligned_events)}")
        
        if aligned_events:
            # Count events by type in range
            event_types_range = {}
            for event in aligned_events:
                event_name = event.get('Name', 'Unknown')
                event_types_range[event_name] = event_types_range.get(event_name, 0) + 1
            
            print(f"\n  Event types in extracted range:")
            for event_name, count in sorted(event_types_range.items(), key=lambda x: x[1], reverse=True):
                print(f"    {event_name}: {count}")
    
    print("=" * 70)
    print("\nVariables created:")
    print("  - sleep_stages: All sleep stages (list)")
    print("  - sleep_stage_times: Time axis for sleep stages (seconds)")
    print("  - aligned_sleep_stages: Sleep stages in extracted time range")
    print("  - aligned_sleep_stage_times: Time axis for aligned sleep stages")
    print("  - scored_events: All scored events (list of dicts)")
    print("  - events_df: Scored events as DataFrame")
    print("  - aligned_events: Events in extracted time range")
    print("  - epoch_length: Epoch length in seconds (typically 30)")
    print("=" * 70)
    
else:
    print(f"⚠️  XML file not found at: {xml_file}")
    print("Cannot load annotations")

XML ANNOTATION READING

Loading XML file: /Users/yangqi/Desktop/RBD/luzy_raw_data/C067/C067_EDF_Export.edf.XML
File exists: True

Epoch length: 30.0 seconds
Sleep stages found: 1215 epochs
  Total duration: 10.12 hours
  Time range: 0 - 36450.0 seconds

Scored events found: 606 events

Event types summary:
  SpO2 desaturation: 186
  Hypopnea: 181
  Arousal (ARO RES): 137
  Arousal (ARO SPONT): 90
  Obstructive Apnea: 8
  SpO2 artifact: 4

Events DataFrame created: (606, 6)

First few events:
                  Name    Start   Duration           Input  LowestSpO2  Desaturation
0        SpO2 artifact  4976.54  615.86000            SpO2         NaN           NaN
1  Arousal (ARO SPONT)  6285.76    7.31859              C4         NaN           NaN
2  Arousal (ARO SPONT)  6318.24   13.46990              C4         NaN           NaN
3             Hypopnea  6369.93   14.67630  Nasal Pressure         NaN           NaN
4    SpO2 desaturation  6378.13   48.56120            SpO2        94.0        